Import librairies and packages

In [1]:
from tdmclient import ClientAsync, aw

import os
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import Vision.CamCalib as calib
import Vision.EnvTracker as tracker
import GlobalNav.GlobalNavHelper as d_star
import time
import cv2
import threading
import importlib


importlib.reload(d_star)
importlib.reload(tracker)

sys.path.insert(0, os.path.join(os.getcwd(), 'LocalNav/sensors'))
from local_occupancy import thymio_coords

sys.path.insert(0, os.path.join(os.getcwd(), 'Kalman'))
from Kalman import Kalman

sys.path.insert(0, os.path.join(os.getcwd(), 'plot'))
from plot import plot

sys.path.insert(0, os.path.join(os.getcwd(), 'NMPC'))
from nmpc import nmpc
from find_limits import find_limits

sys.path.insert(0, os.path.join(os.getcwd(), 'Motion'))
from PdControl import pd_control

sys.path.insert(0, os.path.join(os.getcwd(), 'LocalNav'))
sys.path.insert(0, os.path.join(os.getcwd(), 'GlobalNav'))
from LocalNav import localNav
#from GlobalNav import GlobalNav
%matplotlib tk

c:\Users\alexa\anaconda3\envs\mobile_robotics\lib\site-packages\do_mpc\sysid\__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
c:\Users\alexa\anaconda3\envs\mobile_robotics\lib\site-packages\do_mpc\opcua\__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


Initialize components & objects

In [2]:
#Start Thymio
USE_THYMIO = True

if USE_THYMIO:
    #Create a Client Object:
    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()

    client.process_waiting_messages()

In [3]:
# Parameters
MAP_W, MAP_H = (120, 120)  # In cm

# Initialize camCalib object to retrieve camera parameters
cam_calib = calib.CamCalib(None, (16,9))
cam_mat, dist_coefs, rvecs, tvecs = cam_calib.load_camera_params()
quit = False
run = True
wait_for_map = True
map_created = False

# Initialize the environment tracker object to create global map and track thymio
env_tracker = tracker.EnvTracker((MAP_W, MAP_H), cam_mat, dist_coefs)

map_global = np.zeros((MAP_W, MAP_H))

NEXT_TARGET_DIST = 4
KIDNAPPING_DIST = 20

def set_motors(left_speed, right_speed):
    v = {
        "motor.left.target": [left_speed],
        "motor.right.target": [right_speed],
    }
    if USE_THYMIO:
        aw(node.set_variables(v))

set_motors(0,0)


Main loop (run thymio)

In [4]:
# Open camera feed & initialize map
cv2.destroyAllWindows()
cv2.namedWindow("Thymio Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Thymio Tracking", 1680, 1050)
cam_feed = cv2.VideoCapture(0)
cam_feed.set(cv2.CAP_PROP_FRAME_WIDTH, 1680)
cam_feed.set(cv2.CAP_PROP_FRAME_HEIGHT, 1050)
if not cam_feed.isOpened():
    print("ERROR :: Could not open camera video feed!")
map_created, map_global, goal_pose, thymio_init_pose = env_tracker.wait_for_map(cam_feed, cam_calib, window_name="Thymio Tracking")

print("Thymio init pose: ", thymio_init_pose)
print("GOAL: ", goal_pose)

# Switch map X,Y
map_global = np.transpose(map_global)

# Create figure to show the plots
plt.show()
quit = False
frame_count = 0
fig_main, axs = plt.subplots(2,2, height_ratios=[2,1])
(ax_main, ax_dstar) = axs[0,:]
(ax_info, ax_extra) = axs[1,:]
fig_main.set_size_inches((9,7))
ax_main.set_title('Global View')
ax_dstar.set_title('D* Best Path')
ax_info.set_title('Thymio Infos')

# GLOBAL PLANNING 
if np.sum(goal_pose) >= 0 and np.sum(thymio_init_pose[:2]) > 0:
    # If valid pose for goal and thymio
    best_path = d_star.FindGlobalPath(tuple(thymio_init_pose[:2].astype(int)), tuple(goal_pose.astype(int)), map_global.T, map_global.T, ax_dstar)

# Adjustments before tracking
track_thymio = True
show_markers = False
frame_counter = 0
text_pose = 'POS: ??'
text_angle = 'ANGLE: ??'
text_vel = 'VEL: ??'
text_ang_vel = 'ANG. VEL: ??'
text_goal = f'GOAL: {np.round(goal_pose, 1)}'
pose_hist = np.zeros((2,10))
angle_hist = np.zeros(10)
cv2.resizeWindow("Thymio Tracking", 600, 600)

reach_goal = False
on_goal = False
thymio_detected = False
reverse = False
L = []
T = []
abs_pos = np.array([thymio_init_pose])

optimal_state_estimate_k = np.array([0.0,0.0,0.0])
covariance_estimate_k = np.array([[0.1,0,0],[0,0.1,0],[0,0,0.1]]) #à modifier
L.append(optimal_state_estimate_k)
T.append(covariance_estimate_k)

controller_r = 0
controller_l = 0
next_point_index = min(NEXT_TARGET_DIST+1, best_path.shape[1] - 1)
dest_point = np.array([[best_path[0,next_point_index], best_path[1,next_point_index]]])
while run and not quit:
    # -------------
    # VISION
    # ------------
    # Acquire cam image and perform thymio tracking
    success, frame = cam_feed.read()
    if not success:
        run = False
    # Detect map & thymio
    frame_corrected, roi = cam_calib.undistord(frame, cam_mat, dist_coefs)
    x,y,w,h = roi
    frame_corrected = frame_corrected[y:y+h, x:x+w, :]
    if frame_count % 100:
        env_tracker.detectMarkers(frame_corrected)
        env_tracker.updateMapROI(frame_corrected)
    if track_thymio:
        ax_extra.clear()
        ax_extra.axis('off')
        thymio_detected, thymio_pose, thymio_vel, img_thymio = env_tracker.updateThymio(frame_corrected, show_markers)
        img_sidebar = np.zeros((img_thymio.shape[0], 200, 3), dtype=np.uint8)
        if thymio_detected:
            abs_pos = np.array([thymio_pose])
            ax_info.clear()
            ax_info.axis('off')
            ax_info.plot([70,0,0,70,70], [0,0,70,70,0])
            text_pose = f'POSE: {np.round(thymio_pose[:2], 1)} cm'
            text_angle = f'ANGLE: {round(thymio_pose[2]*180/math.pi, 0)} °'
            text_vel = f'VEL: {np.round(thymio_vel[:2], 1)} cm/s'
            text_ang_vel = f'ANG. VEL: {round(thymio_vel[2]*180/math.pi, 1)} °/s'
            text_goal = f'Dist. to goal: {round(np.linalg.norm(thymio_pose[:2] - goal_pose), 0)} cm'
            ax_info.text(5,60,text_pose)
            ax_info.text(5, 50, text_angle)
            ax_info.text(5,40, text_vel)
            ax_info.text(5, 30, text_ang_vel)
            ax_info.text(5,20,text_goal)
            ax_info.text(5,10,f'Goal reached: {on_goal}')
    else:
        img_thymio = env_tracker.getProjectedMap(frame_corrected)
    # Get latest goal pose
    if np.linalg.norm(np.array([best_path[0,-1], best_path[1,-1]]) -  env_tracker._goal_pose) > 5:
        # Update the path if goal changes
        print("RECALCULATING GOAL...")
        goal_pose = env_tracker._goal_pose
        best_path = d_star.FindGlobalPath(tuple(abs_pos[0][:2].astype(int)), tuple(goal_pose.astype(int)), map_global.T, temp.T, ax_dstar)
        next_point_index = min(10, best_path.shape[1] - 1)
        dest_point = np.array([[best_path[0,next_point_index], best_path[1,next_point_index]]])
        
    cv2.imshow("Thymio Tracking",img_thymio)
    key = cv2.waitKey(1)
    if key == ord('q'):
        run = False
    elif key == ord('m'):
        show_markers = True
    
    if USE_THYMIO:
        await node.wait_for_variables({"prox.horizontal"})
        prox_horizontal = list(node.v.prox.horizontal)
    else:
        prox_horizontal = np.zeros(7)


    #------------------
    # LOCAL NAV +
    # OBSTACLES AVOIDANCE
    #------------------
    temp = map_global.copy()
    map_global = localNav(np.array([optimal_state_estimate_k]), np.array([goal_pose[:]]), prox_horizontal, map_global)

    if not np.array_equal(map_global,temp) or reverse:
        # Go back for the pose
        if not reverse: 
            set_motors(0,0)
            print("RECALCULATING PATH....")
        try:
            best_path = d_star.FindGlobalPath(tuple(optimal_state_estimate_k[:2].astype(int)), tuple(goal_pose.astype(int)), map_global.T, temp.T, ax_dstar)
            reverse = False
            next_point_index = min(NEXT_TARGET_DIST, best_path.shape[1] - 1)
            dest_point = np.array([[best_path[0,next_point_index], best_path[1,next_point_index]]])
        except:
            reverse = True
            set_motors(-100,-100)
        
    # -----------
    # FILTERING
    # ----------
    optimal_state_estimate_k,covariance_estimate_k = Kalman(abs_pos[0],[math.sqrt(thymio_vel[0]**2+thymio_vel[0]**2), thymio_vel[0]],L[-1],T[-1])
    L.append(optimal_state_estimate_k)
    T.append(covariance_estimate_k)

    # --------
    # MOTION
    # -------
    controls = pd_control(optimal_state_estimate_k, dest_point[0])
    controller_l, controller_r = (controls[0], controls[1])
    # Used to get closer if next target point is the goal
    if reach_goal:
        min_dist = 2
    else:
        min_dist = NEXT_TARGET_DIST
    dist_to_tracker = np.linalg.norm(optimal_state_estimate_k[:2] - dest_point[0])
    if not reverse:
        if dist_to_tracker > KIDNAPPING_DIST:
            # Thymio was kidnapped, recompute global path
            print("THYMIO KIDNAPPED!")
            set_motors(0,0)
            on_goal = False
            best_path = d_star.FindGlobalPath(tuple(optimal_state_estimate_k[:2].astype(int)), tuple(goal_pose.astype(int)), map_global.T, temp.T, ax_dstar)
            next_point_index = min(NEXT_TARGET_DIST+1, best_path.shape[1] - 1)
            dest_point = np.array([[best_path[0,next_point_index], best_path[1,next_point_index]]])
        
        elif dist_to_tracker > min_dist:
            # Update motor command to track the next point
            left_m = int(controller_l) * 1
            right_m = int(controller_r) * 1
            set_motors(left_m, right_m)
            #print(f'Left motor: {left_m} | Right motor: {right_m}')
        else:
            # GO to next point
            if reach_goal:
                set_motors(0,0)
                on_goal = True
            next_point_index = min(next_point_index + NEXT_TARGET_DIST, best_path.shape[1] - 1)
            reach_goal = next_point_index == (best_path.shape[1] - 1)
            dest_point = np.array([[best_path[0,next_point_index], best_path[1,next_point_index]]])
    
    # -----------
    # PLOTTING
    # -----------
    if frame_count % 1 == 0:
        ax_main.clear()
        ax_info.text(5,1,f'Next target: {dest_point[0]}')
        plot(abs_pos, thymio_coords, np.array([goal_pose[:]]), map_global, abs_pos[0][2], 
             0, 0, 0, 0, ax_main, best_path=best_path, next_target=dest_point[0])
        fig_main.canvas.draw()
        fig_main.canvas.flush_events()

    frame_count += 1

if cam_feed:
    cam_feed.release()
cv2.destroyAllWindows()
plt.close('all')

set_motors(0,0)

Pixels per grid cell: 6.533333333333333 x 4.641666666666667
Goal detected @:  [85.791 25.153]
Thymio init pose:  [ 64 102   1]
GOAL:  [85.791 25.153]
going with this start:  (64, 102)
initialization
state =  1
path found !
RECALCULATING PATH....
going with this start:  (63, 101)
update
state =  2
path found !
RECALCULATING PATH....
going with this start:  (63, 99)
update
state =  2
path found !
RECALCULATING PATH....
going with this start:  (62, 98)
initialization
state =  1
path found !
RECALCULATING PATH....
going with this start:  (62, 98)
update
state =  2
path found !
RECALCULATING PATH....
going with this start:  (29, 83)
update
state =  2
path found !
RECALCULATING PATH....
going with this start:  (29, 82)
update
state =  2
path found !
RECALCULATING PATH....
going with this start:  (29, 80)
update
state =  2
path found !
RECALCULATING PATH....
going with this start:  (29, 80)
update
state =  2
path found !
RECALCULATING PATH....
going with this start:  (29, 79)
update
state =  

In [5]:
cam_feed.release()
set_motors(0,0)
if USE_THYMIO:
    await node.unlock()